### Step1: Import packages

In [1]:
import torch
import os
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoModelForPreTraining, pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration

### Step2: Read dataset

In [2]:
ds = load_dataset("cnn_dailymail", "1.0.0")
ds = ds.shuffle(seed=42)
ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [31]:
ds['train'][0]

{'article': "By . Anthony Bond . PUBLISHED: . 07:03 EST, 2 March 2013 . | . UPDATED: . 08:07 EST, 2 March 2013 . Three members of the same family who died in a static caravan from carbon monoxide poisoning would have been unconscious 'within minutes', investigators said today. The bodies of married couple John and Audrey Cook were discovered alongside their daughter, Maureen, at the mobile home they shared on Tremarle Home Park in Camborne, west Cornwall. The inquests have now opened into the deaths last Saturday, with investigators saying the three died along with the family's pet dog, of carbon monoxide poisoning from a cooker. Tragic: The inquests have opened into the deaths of three members of the same family who were found in their static caravan last weekend. John and Audrey Cook are pictured . Awful: The family died following carbon monoxide poisoning at this caravan at the Tremarle Home Park in Camborne, Cornwall . It is also believed there was no working carbon monoxide detect

### Step2.5: Check GPU is available

In [3]:
torch.cuda.is_available()

True

### Step3: Analyze data

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("./summary/last-checkpoint-10240", max_seq_len=1024)
#tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small", max_length=1024)

In [ ]:
def process_func(examples):
    contents = ['Generate summary: \n' + e for e in examples['article']]
    inputs = tokenizer(contents, max_length=1024, truncation=True)
    labels = tokenizer(text_target=examples['highlights'], max_length=64, truncation=True)
    inputs['labels'] = labels['input_ids']
    return inputs

In [34]:
tokenized_ds = ds.map(process_func, batched=True)
tokenizer.decode(tokenized_ds['train'][0]['input_ids'])

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [32]:
tokenizer.decode(tokenized_ds['train'][0]['labels'])

'John and. Audrey Cook were discovered alongside their daughter, Maureen. They were found at Tremarle Home Park in Cornwall. Investigators say the three died of carbon monoxide. poisoning.</s>'

In [33]:
ds['train'][0]['highlights']

'John and . Audrey Cook were discovered alongside their daughter, Maureen . They were found at Tremarle Home Park in Cornwall . Investigators say the three died of carbon monoxide . poisoning .'

### Step4: Create model

In [36]:
#model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small", max_memory = 1024)
model = T5ForConditionalGeneration.from_pretrained("./summary/last-checkpoint", max_memory = 1024)
print("Model weights loaded...\n")

Model weights loaded...


### Step5: Create evaluate function using rouge

In [37]:
import numpy as np
from rouge import Rouge

rouge = Rouge()

def compute_metric(evalPred):
    preds, labels = evalPred
    decode_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decode_preds = [" ".join(p) for p in decode_preds]
    decode_labels = [" ".join(p) for p in decode_labels]
    scores = rouge.get_scores(decode_preds, decode_labels, avg=True)
    return {
        "rouge-1": scores['rouge-1']['f'],
        "rouge-2": scores['rouge-2']['f'],
        "rouge-l": scores['rouge-l']['f']
    }

### Step6: Set training parameters

In [38]:
args = Seq2SeqTrainingArguments(
    output_dir="./summary",
    learning_rate=1e-6,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    warmup_steps=128,
    logging_steps=512,
    logging_dir="./logging",
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=512,
    save_total_limit=5,     # save the last 5 model
    metric_for_best_model="rouge-l",
    predict_with_generate=True,  # must set True
    #load_best_model_at_end=True
)

### Step7: Create trainer

In [ ]:
trainer = Seq2SeqTrainer(
    args=args,
    model=model,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    compute_metrics=compute_metric,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
)

### Step8: Train the model

In [30]:
trainer.train()

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
512,1.696300,1.619595,0.773501,0.415039,0.694320


Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
512,1.696300,1.619595,0.773501,0.415039,0.694320


Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
512,1.696300,1.619595,0.773501,0.415039,0.694320
1024,1.696600,1.620052,0.773526,0.415000,0.694275


Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
512,1.696300,1.619595,0.773501,0.415039,0.694320
1024,1.696600,1.620052,0.773526,0.415000,0.694275


KeyboardInterrupt: 

### Step9: Evaluate the T5 model

In [57]:
pipe = pipeline('text2text-generation', model=model, tokenizer=tokenizer, device=0)
text = ds['validation'][1000]['article']
print(text)
print('-'*20)
target = ds['validation'][1000]['highlights']
print(target)
print('-'*20)
pip_res = pipe("Generate summary:\n" + text, max_length = 64)
t5_summary = pip_res[0]['generated_text']
print(t5_summary)
print('-'*20)
print(" Rouge-L between label and generate summary with t5 model is ", rouge.get_scores(target, t5_summary)[0]['rouge-l'])

As fans continue to mourn the passing of their favorite Vulcan, Leonard Nimoy's passion  for photography - and the women he put in front of the lens - is also being recognized. Nimoy, called the 'Conscience of Star Trek' by the show's creator, once said he used photography to express the idea of feminine power. After eight years of taking photographs of plus-sized women, Nimoy published a collection titled The Full Body Project in 2007, which featured obese women photographed in the nude. It was when Nimoy was doing an exhibit for a different photography collection that he was turned to the idea of shooting fuller-figured women. After eight years of taking photographs of plus-sized women, Leonard Nimoy published a collection titled The Full Body Project in 2007, which featured obese women photographed in the nude . For The Full Body Project, Nimoy found new subjects in the plus-size burlesque group The Fat-Bottom Revue . He was inspired by their late founder Heather MacAllister, who he

### Step10: Sentence ranking method

In [50]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
from datasets import load_dataset
from rouge import Rouge
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

stopwords = list(STOP_WORDS)
nlp = spacy.load('en_core_web_sm')

In [51]:
def select_main_sentence(text, punctuation, nlp):
    summary_length = 3
    doc = nlp(text)
    tokens = [token.text for token in doc]
    punctuation = punctuation + '\n'
    sentence_tokens = [sent for sent in doc.sents]
    
    word_frequencies = {}
    for word in doc:
        if word.text.lower() not in stopwords:
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1

    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent] += word_frequencies[word.text.lower()]
         
    summary = nlargest(summary_length, sentence_scores, key = sentence_scores.get)
    return summary
    
    

In [56]:
print("\n----------------------------article---------------------------------------\n")
text = ds['validation'][1000]['article']
print(text)
print("\n----------------------------label---------------------------------------\n")
target = ds['validation'][1000]['highlights']
print(target)
print("\n----------------------------sentence ranking---------------------------------------")
summary = select_main_sentence(text, punctuation, nlp)
generate_summary = ""
for each in summary:
    generate_summary = generate_summary + str(each)
print(generate_summary)
print(" Rouge-L: ", rouge.get_scores(target, generate_summary)[0]['rouge-l'])

print("\n----------------------------fine-tuned t5 model---------------------------------------")
pip_res = pipe("Generate summary:\n" + text, max_length = 64)
t5_summary = pip_res[0]['generated_text']
print(t5_summary)
print(" Rouge-L between label and generate summary with t5 model is ", rouge.get_scores(target, t5_summary)[0]['rouge-l'])


----------------------------article---------------------------------------

As fans continue to mourn the passing of their favorite Vulcan, Leonard Nimoy's passion  for photography - and the women he put in front of the lens - is also being recognized. Nimoy, called the 'Conscience of Star Trek' by the show's creator, once said he used photography to express the idea of feminine power. After eight years of taking photographs of plus-sized women, Nimoy published a collection titled The Full Body Project in 2007, which featured obese women photographed in the nude. It was when Nimoy was doing an exhibit for a different photography collection that he was turned to the idea of shooting fuller-figured women. After eight years of taking photographs of plus-sized women, Leonard Nimoy published a collection titled The Full Body Project in 2007, which featured obese women photographed in the nude . For The Full Body Project, Nimoy found new subjects in the plus-size burlesque group The Fat-Bot

### Step11: Comparison between different models

In [63]:
original_model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small", max_memory = 1024)
original_pipe = pipeline('text2text-generation', model=original_model, tokenizer=tokenizer)


In [91]:
validations = ds['validation'].shuffle(seed=42)
validations = validations[:int(.1*len(validations))]
texts = validations['article']
labels = validations['highlights']


In [92]:
rouge_ranking = []
rouge_original = []
rouge_t5 = []
for idx in range(len(texts)):
    n = len(texts)
    text = texts[idx]
    target = labels[idx]
    summary_ranking = select_main_sentence(text, punctuation, nlp)
    generate_summary = ""   
    for each in summary_ranking:
        generate_summary = generate_summary + str(each)
        
    rouge_ranking.append(rouge.get_scores(target, generate_summary)[0]['rouge-l'])  
    
    pip_res = pipe("Generate summary:\n" + text, max_length = 64)
    t5_summary = pip_res[0]['generated_text']
    rouge_t5.append(rouge.get_scores(target, t5_summary)[0]['rouge-l'])
    
    original_res = original_pipe("Generate summary:\n" + text, max_length = 64)
    original_summary = original_res[0]['generated_text']
    rouge_original.append(rouge.get_scores(target, original_summary)[0]['rouge-l'])
    print("\r{}/{}".format(idx+1,n), end="")
    
    

1336/1336

In [97]:
def compute_avg(score):
    n = len(score)
    r = 0
    p = 0
    f = 0
    for each in score:
        r = r + each['r']
        p = p + each['p']
        f = f + each['f']
    return {'r':r/n, 'p':p/n,'f': f/n}      

In [98]:
compute_avg(rouge_t5)

{'r': 0.4069127773617567, 'p': 0.3150771264055881, 'f': 0.3470146897310208}

In [99]:
compute_avg(rouge_ranking)

{'r': 0.22760192852431332, 'p': 0.3474619074765952, 'f': 0.26578509271083867}

In [100]:
compute_avg(rouge_original)

{'r': 0.3484716288736835, 'p': 0.24377807127441672, 'f': 0.27862577607101885}